# Detention By Nationality Analysis

The full methodology for this analysis is available [here](../methodology.md).

## Load the data

In [2]:
import pandas as pd
import sys
sys.path.append("../utils")
import loaders

*Note: loaders is a custom module to handle basic data-loading. It is available [here](https://github.com/BuzzFeedNews/2015-08-immigration/blob/master/utils/loaders.py).*

In [7]:
first_scheduled_proceeding = pd.read_csv("../data/first-scheduled-proceeding.csv", 
     parse_dates=["ADJ_DATE"],
     dtype={
          "IDNCASE": str,
          "IDNPROCEEDING": str,
     },
     encoding='latin1'
)

*Note: first-scheduled-proceeding.csv is a pre-processed data file. The code to create that file from tbl_schedule.csv is available [here](../utils/generate-first-scheduled-proceeding.py).*

In [27]:
nationality_table = loaders.load_file("tblLookupNationality.csv")

/var/folders/1g/crb4nhbs103_dd6qrzt577x80000gn/T/ipykernel_31227/2219749348.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  nationality_table = loaders.load_file("tblLookupNationality.csv")


In [4]:
case_date_list = [
    "E_28_DATE",
    "DATE_OF_ENTRY",
    "C_BIRTHDATE",
    "C_RELEASE_DATE",
    "DATE_DETAINED",
    "DATE_RELEASED"
]

In [5]:
_cases = loaders.load_file("A_tblCase.csv",
    parse_dates=case_date_list,
    dtype={
        "IDNCASE": str
    },
)

/var/folders/1g/crb4nhbs103_dd6qrzt577x80000gn/T/ipykernel_31227/2101931097.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  _cases = loaders.load_file("A_tblCase.csv",


In [8]:
_cases["GENDER"] = _cases["GENDER"].fillna("UNK")

In [9]:
_charges = loaders.load_file("B_tblProceedCharges.csv",
    dtype={ "IDNCASE": str, "IDNPROCEEDING": str })

/var/folders/1g/crb4nhbs103_dd6qrzt577x80000gn/T/ipykernel_31227/553268753.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  _charges = loaders.load_file("B_tblProceedCharges.csv",
b'Skipping line 1165848: expected 5 fields, saw 6\n'
b'Skipping line 1433634: expected 5 fields, saw 6\n'
b'Skipping line 2646392: expected 5 fields, saw 6\n'
b'Skipping line 2847501: expected 5 fields, saw 6\n'
b'Skipping line 2947399: expected 5 fields, saw 6\n'
b'Skipping line 3131015: expected 5 fields, saw 6\n'


*Note: Six rows — of the more than 8 million total rows — in the charges table contain malformed data stemming from extra tab characters, triggering the warning messages above.*

## Process the data

Join the various tables and prepare them for analysis.

In [10]:
charges_group = _charges.groupby([ "IDNCASE", "IDNPROCEEDING" ])

In [11]:
charge_lists = pd.DataFrame({
    "charge_list": charges_group["CHARGE"].apply("|".join)
}).reset_index()

In [12]:
charge_lists.head()

,IDNCASE,IDNPROCEEDING,charge_list
0,2046920,3200048,212a06Ai
1,2046921,3200049,212a06Ai
2,2046922,3200050,212a06Ai
3,2046923,3200051,212a06Ci
4,2046923,3525150,212a06Ci


In [13]:
assert(charge_lists["IDNCASE"].nunique() == 5033293)
assert(len(first_scheduled_proceeding) == 5045511)

From the numbers above: A small fraction of cases — approximately 0.2% — have a scheduled proceding but no charges.

In [14]:
cases_with_first_proceeding = first_scheduled_proceeding\
    .merge(charge_lists, how="left", on=[ "IDNCASE", "IDNPROCEEDING" ])\
    .merge(_cases, how="left", on="IDNCASE", suffixes=["_schedule", "_case"])

Legal representatives file the EOIR-28 form to notify the court of their representation for a given immigrant.

`ADJ_DATE` in this table indicates the date of the case's first proceeding.

In [15]:
cases_with_first_proceeding["legal_rep_at_first_proceeding"] = cases_with_first_proceeding\
    .apply(lambda x: x["E_28_DATE"] <= x["ADJ_DATE"], axis=1)

## Select non-criminal removal cases between Jan. 1, 2003 and Jan. 1, 2015

In [185]:
selected_cases = cases_with_first_proceeding[
     # Select cases with first-scheduled-hearing dates in 2003–2014
    (cases_with_first_proceeding["ADJ_DATE"] >= "2003-01-01") &
    (cases_with_first_proceeding["ADJ_DATE"] < "2015-01-01")
].copy()

KeyboardInterrupt: 

In [ ]:
selected_cases["has_criminal_charge"] = (
    selected_cases["charge_list"].str.contains("237a02") |
    selected_cases["charge_list"].str.contains("212a02")
)

In [ ]:
selected_cases["detained"] = selected_cases["CUSTODY"].map({"N": 0, "D": 1, "R": 1})

In [ ]:
selected_cases['year'] = pd.DatetimeIndex(selected_cases["ADJ_DATE"]).year

## Calculate detention rates by nationality

In [197]:
custody_by_nationality = selected_cases.groupby(["NAT", "year", "CUSTODY", "has_criminal_charge"])\
    .size()\
    .unstack()\
    .unstack()\
    .fillna(0)

In [198]:
def column_map(col):
    if col[0]:
        return f'{col[1]}_criminal'
    else:
        return f'{col[1]}_noncriminal'

In [199]:
custody_by_nationality.columns = [column_map(col) for col in custody_by_nationality.columns]

In [200]:
custody_by_nationality = custody_by_nationality.reset_index()

In [201]:
custody_by_nationality["total_criminal"] = custody_by_nationality[['N_criminal', 'D_criminal', 'R_criminal']].sum(axis=1)
custody_by_nationality["total_noncriminal"] = custody_by_nationality[['N_noncriminal', 'D_noncriminal', 'R_noncriminal']].sum(axis=1)

In [202]:
custody_by_nationality["percent_detained_criminal"] = custody_by_nationality\
    .apply(lambda x: None if x['total_criminal'] == 0 else round(100.0 * (x["D_criminal"] + x["R_criminal"]) / x["total_criminal"], 1), axis=1)

custody_by_nationality["percent_detained_noncriminal"] = custody_by_nationality\
    .apply(lambda x: None if x['total_noncriminal'] == 0 else round(100.0 * (x["D_noncriminal"] + x["R_noncriminal"]) / x["total_noncriminal"], 1), axis=1)

In [203]:
# Add full country names
custody_by_nationality = pd.merge(
    custody_by_nationality,
    nationality_table[['NAT_CODE', 'NAT_NAME']],
    how='left',
    left_on=['NAT'],
    right_on=['NAT_CODE'],
)

In [207]:
main_columns = ["NAT_NAME", "year",\
    "N_criminal", "D_criminal", "R_criminal", "total_criminal", "percent_detained_criminal",
    "N_noncriminal", "D_noncriminal", "R_noncriminal", "total_noncriminal", "percent_detained_noncriminal",
]
custody_by_nationality = custody_by_nationality.sort_values(["NAT_NAME", "year"])[main_columns]

## Table: Per-Nationality Detention Rate

In [208]:
custody_by_nationality

,NAT_NAME,year,N_criminal,D_criminal,R_criminal,total_criminal,percent_detained_criminal,N_noncriminal,D_noncriminal,R_noncriminal,total_noncriminal,percent_detained_noncriminal
38,AFGHANISTAN,2003,10.0,26.0,13.0,49.0,79.6,102.0,5.0,14.0,121.0,15.7
39,AFGHANISTAN,2004,10.0,31.0,1.0,42.0,76.2,64.0,20.0,7.0,91.0,29.7
40,AFGHANISTAN,2005,5.0,33.0,3.0,41.0,87.8,34.0,4.0,8.0,46.0,26.1
41,AFGHANISTAN,2006,7.0,19.0,5.0,31.0,77.4,37.0,7.0,7.0,51.0,27.5
42,AFGHANISTAN,2007,3.0,27.0,4.0,34.0,91.2,47.0,6.0,4.0,57.0,17.5
...,...,...,...,...,...,...,...,...,...,...,...,...
0,NaN,2003,1.0,3.0,1.0,5.0,80.0,228.0,447.0,17.0,692.0,67.1
1,NaN,2004,0.0,3.0,0.0,3.0,100.0,118.0,206.0,9.0,333.0,64.6
1637,NaN,2005,0.0,1.0,0.0,1.0,100.0,0.0,0.0,0.0,0.0,NaN
2,NaN,2009,0.0,0.0,0.0,0.0,NaN,0.0,1.0,0.0,1.0,100.0


In [209]:
# save to output dataframe
custody_by_nationality.to_csv("custody_by_nationality_year.csv", index=False)